## **Limpieza del CSV de "Humedad del aire a 2 metros"**
----
**Elaborado con base en**
* [Humedad del Aire 2 metros | Datos Abiertos Colombia](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Humedad-del-Aire-2-metros/uext-mhny/about_data)

#### ***Importación de paquetes***

In [50]:
import pandas as pd

#### ***Cargar el CSV mediante Pandas***

In [51]:
df = pd.read_csv("../CSV/Humedad_del_Aire_2_metros_20240613.csv", low_memory=False)
df

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0057025020,27.0,06/28/2008 04:00:00 PM,88.00000,GORGONA GUAPI,CAUCA,GUAPI,GUAVIARE,2.963000,-78.174,Humedad del aire 2 mt,%
1,0053045040,27.0,12/03/2012 01:00:00 AM,96.00000,AEROPUERTO GUAPI - AUT,CAUCA,GUAPI,TAPAJE - DAGUA - DIRECTOS,2.574417,-77.89475,Humedad del aire 2 mt,%
2,0021055070,27.0,04/16/2012 08:00:00 PM,85.00000,INZA - AUT,CAUCA,INZÁ,ALTO MAGDALENA,2.548194,-76.06394444,Humedad del aire 2 mt,%
3,0052025080,27.0,07/15/2014 03:00:00 AM,79.00000,ESTRECHO PATIA - AUT,CAUCA,PATÍA (EL BORDO),PATÍA,1.962778,-77.12027778,Humedad del aire 2 mt,%
4,0052025501,27.0,05/22/2018 07:55:00 PM,87.13305,LA VEGA - AUT,CAUCA,LA VEGA,PATÍA,2.092942,-76.81202778,Humedad del aire 2 mt,%
...,...,...,...,...,...,...,...,...,...,...,...,...
8936971,0053115502,27.0,06/01/2020 12:20:00 PM,89.37495,JULIO FERNANDEZ - AUT,VALLE DEL CAUCA,RESTREPO,TAPAJE - DAGUA - DIRECTOS,3.809164,-76.5330{,NaN,NaN
8936972,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8936973,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8936974,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### ***Liberarnos de ciertas columnas***

In [52]:
df = df.drop(["CodigoSensor", "CodigoEstacion", "NombreEstacion", "DescripcionSensor", "UnidadMedida"], axis=1)
df.head()

,FechaObservacion,ValorObservado,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud
0,06/28/2008 04:00:00 PM,88.00000,CAUCA,GUAPI,GUAVIARE,2.963000,-78.174
1,12/03/2012 01:00:00 AM,96.00000,CAUCA,GUAPI,TAPAJE - DAGUA - DIRECTOS,2.574417,-77.89475
2,04/16/2012 08:00:00 PM,85.00000,CAUCA,INZÁ,ALTO MAGDALENA,2.548194,-76.06394444
3,07/15/2014 03:00:00 AM,79.00000,CAUCA,PATÍA (EL BORDO),PATÍA,1.962778,-77.12027778
4,05/22/2018 07:55:00 PM,87.13305,CAUCA,LA VEGA,PATÍA,2.092942,-76.81202778


#### ***Enfocarnos en los departamentos de la región Pacífica***

In [53]:
#df.head()
df["Departamento"].value_counts()

Departamento
VALLE DEL CAUCA    3502675
NARIÑO             2178996
CAUCA              2133539
CHOCO               648614
CHOCÓ               445428
NARINO               27720
Name: count, dtype: int64

#### ***Limpieza en los departamentos***

In [54]:
diccionario_reemplazo = {
    "CHOCO": "CHOCÓ",
    "NARINO": "NARIÑO"
}

df["Departamento"] = df["Departamento"].replace(diccionario_reemplazo)
df["Departamento"].value_counts()

Departamento
VALLE DEL CAUCA    3502675
NARIÑO             2206716
CAUCA              2133539
CHOCÓ              1094042
Name: count, dtype: int64

#### ***Transformar el formato de la fecha y añadir columnas "Año" y "Mes"***

In [55]:
df["FechaObservacion"] = df["FechaObservacion"].astype(str)
df["FechaObservacion"] = df["FechaObservacion"].str.split(' ').str[0]

df["FechaObservacion"] = pd.to_datetime(df["FechaObservacion"])

df["Año"] = df["FechaObservacion"].dt.year
df["Mes"] = df["FechaObservacion"].dt.month

df["FechaObservacion"] = df["FechaObservacion"].dt.date

In [56]:
df = df.dropna()
df = df[df != '']

if df is not None:
    df['Año'] = df['Año'].astype(int, errors='ignore')
    df['Mes'] = df['Mes'].astype(int, errors='ignore')

# Reorganizar las columnas
cols = ['Año', 'Mes'] + [col for col in df.columns if col not in ['Año', 'Mes']]
df = df[cols]

# Organizar los años y los meses de manera ascendente
df = df.sort_values(by=['Año', 'Mes'], ascending=[True, True])

#### ***Eliminar datos que no se encuentren en el rango de año de 2005 a 2023***

In [57]:
df = df[(df['Año'] >= 2005) & (df['Año'] <= 2023)]
df["Año"].value_counts()

Año
2019    1633556
2018    1431195
2017    1333443
2020     865499
2016     692964
2021     400154
2022     370382
2023     340373
2015     203759
2014     199586
2009     186174
2010     182552
2008     181895
2013     160584
2007     159234
2011     154969
2006     133121
2012     128264
2005      45184
Name: count, dtype: int64

#### ***Guardar la nueva versión del dataset***

In [60]:
df.to_csv("../HumedadAire_RegionPacifico.csv", index=False)